In [9]:
from tkinter import *
from tkinter import ttk, filedialog
from tkinter.filedialog import askopenfilename
import csv
from pathlib import Path
import pymongo
import numpy as np
# Use sentinel for missing values
from numpy import nan as NA
import pandas as pd
# For showing plots in tkinter
import matplotlib.pyplot as plt
from matplotlib.figure import Figure 
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,  
NavigationToolbar2Tk) 
import seaborn as sns

# connect to mongodb
client = pymongo.MongoClient("mongodb://localhost:27017")
# create db
db_formative = client["summative"]
# create collection
collection_vendors = db_formative["test-1-1"]

In [10]:
# Global variables
fileButtons = ["inspections", "inventory", "violations"]
fileRoutes = {fileButtons[0]:"", fileButtons[1]:"", fileButtons[2]:""}

In [11]:
class Parser(object):
    def __init__(self, collection, csvFile):
        self.collection = collection
        self.csvFile = csvFile
        self.dataList = []
        self.dataObj = {}
        self.violationsSet = {}
        # lists and a set (no duplicates in set)
        self.dataDict = {'inspections':[], 'inventory':[], 'violations':[], 'violation codes':set()}
        
    def parseData(self):
        self.readInspections()
        self.readInventory()
        self.readViolations()
        return self.dataDict
                
    def readInspections(self):
        with open(self.csvFile['inspections'], newline="", encoding='utf-8-sig') as inFile: 
            dataReader = csv.DictReader(inFile)
            for row in dataReader:
                # Do not add inactive programs
                if row['PROGRAM STATUS'] == 'INACTIVE':
                    continue
                # Add Mongo ID as Serial Number with INS- prefix
                print(row['SERIAL NUMBER'])
                row['_id'] = "INS-"+row['SERIAL NUMBER']
                # Split PE Description and create new row with seating
                try:
                    start,mid = row["PE DESCRIPTION"].split(' (')
                    mid,end = mid.split(')')
                    row["PE DESCRIPTION"] = start+end
                    row["SEATING DETAILS"] = mid
                except:
                    print('no seating desc')
                # NEW: remove second half of zip (e.g. 90001-1234 remove '1234')
                try:
                    zipStart = row["FACILITY ZIP"].split('-')
                    row["FACILITY ZIP"] = zipStart[0]
                except:
                    print('no zip')
                self.dataDict['inspections'].append(row)
#             print(f'Inspection first doc: \n{self.dataDict['inspections'][0]}')
            ins = self.collection.insert_many(self.dataDict['inspections'])
            print('=> Inspections inserted into DB')

    def readInventory(self):
         with open(self.csvFile['inventory'], newline="", encoding='utf-8-sig') as inFile: 
            dataReader = csv.DictReader(inFile)
            for row in dataReader:
                # Add Mongo ID as Facility ID with F_ID- prefix
                print(row['FACILITY ID'])
                row['_id'] = 'F_ID-'+row['FACILITY ID']
                # Split PE Description and create new row with seating
                start,mid = row["PE DESCRIPTION"].split(' (')
                mid,end = mid.split(')')
                row["PE DESCRIPTION"] = start+end
                row["SEATING DETAILS"] = mid
                self.dataDict['inventory'].append(row)
#             print(f'Inventory first doc: \n{self.dataDict['inventory'][0]}')
            ins = self.collection.insert_many(self.dataDict['inventory'])
            print('=> Inventory inserted into DB')
            
    def readViolations(self):
        with open(self.csvFile['violations'], newline="", encoding='utf-8-sig') as inFile: 
            dataReader = csv.DictReader(inFile)
            # Group the rows by serial number so they can be added to the corresponding inspection
            for row in dataReader:
                # Add violations codes to a set - no duplicates
                self.dataDict['violation codes'].add(row['VIOLATION CODE'])
                if row['SERIAL NUMBER'] in self.dataObj:
                    self.dataObj[row['SERIAL NUMBER']].append(row)
                else:
                    self.dataObj[row['SERIAL NUMBER']] = [row]
            self.dataDict['violations'].append(self.dataObj)
#             print(f'Violations first doc: \n{self.dataDict['violations'][0]}')
            ins = collection_vendors.insert_many(self.dataDict['violations'])
            print('=> Violations inserted into DB')
        
        
    def formatViolations(self):
        for violation in self.dataObj:
            collection_vendors.update_one({'_id': 'INS-'+violation}, {'$set': {'VIOLATIONS': self.dataObj[violation]}})
            

In [12]:
class ButtonUpload(object):
    def __init__(self, fileButtons, fileLabels, counter, mainframe):
        self.fileButtons = fileButtons
        self.fileLabels = fileLabels
        self.counter = counter
        self.mainframe = mainframe
        self.filePath = ''
        self.uploadLabel = StringVar()

    def browseFile(self):
        self.filePath = askopenfilename(filetypes=[("CSV files", ".csv")])
        fileName = Path(self.filePath)
        self.uploadLabel.set(fileName.name)
        print(self.filePath)
        self.setFileRoutes()
    
    def setFileRoutes(self):
        fileRoutes[self.fileButtons] = self.filePath
        print(fileRoutes)
        
    def createButton(self):
        ttk.Label(self.mainframe, text=f"Upload data: {self.fileButtons}", anchor="w").grid(row=self.counter+1, column=0, sticky="w")
        Button(self.mainframe, text="Upload file", command=self.browseFile).grid(row=self.counter+1, column=1, sticky="w")
        ttk.Label(self.mainframe, textvariable=self.uploadLabel).grid(row=self.counter+1, column=2, sticky="w")  

In [13]:
class AveragesCalculator(object):
    def __init__(self, year, category):
        self.year = year
        self.category = category
        
    def zipScores(self):
        print(f'Calculating stats for: \n- Year = {self.year} \n- Category = {self.category}')
        # Find inspection documents - look in ispections (INS-) with year
        resultsInspections = collection_vendors.find({'_id':{'$regex':'INS-'},'ACTIVITY DATE':{'$regex':self.year}})

        # Create dict of facility zips with lists of their scores
        zipScoresDict = {}  
        for r in resultsInspections:
            key = str(r[self.category])
            if(r['SCORE'] == ''):
                value = NA
            else:
                value = int(r['SCORE'])
            if key in zipScoresDict:
                zipScoresDict[r[self.category]].append(value)
            else:
                zipScoresDict[key] = [value]
        return self.calcZipStats(zipScoresDict)


    def calcZipStats(self, zipScoresDict):
        zipScoresMean = {}
        zipScoresMean = {self.category:[],'mean':[],'median':[],'mode':[]}
        for x in zipScoresDict:
            # Create Series to calc mean, mode, median
            tempSeries = pd.Series(zipScoresDict[x])
            mean = tempSeries.mean()
            mode = tempSeries.mode().tolist()
            median = tempSeries.median()

            # mean, mode, median as columns
            zipScoresMean[self.category].append(x)
            zipScoresMean['mean'].append(mean)
            zipScoresMean['median'].append(median)
            zipScoresMean['mode'].append(mode)

        # Create DataFrame
        zipScoresDF = pd.DataFrame(zipScoresMean)
        # sort by ascending cat - easier to find for user (esp. zip)
        zipScoresDF.sort_values(by=[self.category],inplace=True)
        # set cat as index
        zipScores_noIndex = zipScoresDF.set_index(self.category)
        
        return zipScores_noIndex
 

In [14]:
class UserInterface(object):
    def __init__(self, title):
        self.title = title
        self.mainframe = None
        self.window = Tk()
        self.window.title(self.title)
        self.window.geometry("600x400")
        self.window.columnconfigure(0, weight=1)
        self.window.rowconfigure(0, weight=1)
        self.calcCategory = StringVar()
        self.calcYear = StringVar()
        self.violationsDict = None
        self.dataDict = None
        
    def createWindow(self):
        self.createInitialFrame()
        self.window.mainloop()
        
    def createInitialFrame(self):
         # variables
        labelTextOne = StringVar()
        labelTextTwo = StringVar()
        labelTextThree = StringVar()
        fileLabels = [labelTextOne, labelTextTwo, labelTextThree]
        # Config layout
        self.mainframe = ttk.Frame(self.window, padding="16 16 16 16")
        self.mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
        self.mainframe.grid_rowconfigure(4, weight=1)
        self.mainframe.grid_columnconfigure(2, weight=1)
        heading = ttk.Label(self.mainframe, text="Load your dataset", anchor="w").grid(row=0, sticky="nw")
        # Create buttons for file uploads
        for i in range(0, len(fileButtons)): 
            newButton = ButtonUpload(fileButtons[i], fileLabels[i], i, self.mainframe)
            newButton.createButton()
        # Buttons for Cancel and Clean & Save
        footer = ttk.Frame(self.window, padding="16 0 0 0").grid(row=5, column=1)
        cancelButton = Button(self.mainframe, text="Cancel & exit", bg="red").grid(row=4, column=0, sticky="sw")
        saveButton = Button(self.mainframe, text="Save", command=self.handleSave).grid(row=4, column=2,sticky="se")
        
    def handleSave(self):
        self.loadingFrame('Your data is being parsed and loaded into MongoDB...')
        parser = Parser(collection_vendors, fileRoutes)
        self.dataDict = parser.parseData()
        parser.formatViolations()
        self.createNotebook()
    
    def loadingFrame(self, text):
        self.mainframe.destroy()
        self.mainframe = ttk.Frame(self.window, padding="16 16 16 16")
        self.mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
        heading = ttk.Label(self.mainframe, text="Please wait:", anchor="w").grid(row=0, sticky="nw")
        subheading = ttk.Label(self.mainframe, text=text, anchor="w").grid(row=1, sticky="nw")
        
    def createNotebook(self):
        print('here')
        self.mainframe.destroy()
        self.mainframe = ttk.Notebook(self.window)
        f1 = ttk.Frame(self.mainframe)  
        f2 = ttk.Frame(self.mainframe)  
        f3 = ttk.Frame(self.mainframe) 
        self.mainframe.add(f1, text='Averages')
        self.mainframe.add(f2, text='Violations per Establishment')
        self.mainframe.add(f3, text='Correlations')
        # ====== Widget for tab 1
        heading_one = ttk.Label(f1, text="Mean, Mode and Median", anchor="w").grid(row=0, sticky="nw")
        # Radio - Type
        radioHeadingOne = ttk.Label(f1, text="Calculate by seating type or zip codes:", anchor="w").grid(row=0, sticky="nw")
        Radiobutton(f1, text="Vendor seating type", padx = 20, variable=self.calcCategory, value='SEATING DETAILS').grid(row=1, column=0, sticky='w')
        Radiobutton(f1, text="Zip Code", padx = 20, variable=self.calcCategory, value='FACILITY ZIP').grid(row=2, column=0, sticky='w')
        # Radio - Years
        radioHeadingTwo = ttk.Label(f1, text="Select Year", anchor="w").grid(row=3, sticky="nw")
        yearsArr = ['2016', '2017', '2018', '2019']
        for i in range(len(yearsArr)):
            Radiobutton(f1, text=yearsArr[i], padx = 20, variable=self.calcYear, value=yearsArr[i]).grid(row=i+4, column=0, sticky='w')
        # Calc button
        calcButton = Button(f1, text="Calculate", command=self.calcAverages).grid(row=8, column=0, sticky="sw")
        self.mainframe.grid(column=0, row=0, sticky=(N, W, E, S))
        
        # ====== Widget for tab 2
        heading_two = ttk.Label(f2, text="Violations per Establishment", anchor="w").grid(row=0, sticky="nw")
        self.createViolationsPlot(f2)
        
        # ====== Widget for tab 3
        heading_two = ttk.Label(f3, text="Correlation between the number of violations commited per vendor and their zip code", anchor="w").grid(row=0, sticky="nw")
        self.createCorrelationsPlot(f3)
        
    def createViolationsPlot(self, f2):
        # Create dict with violation codes as keys and no. of facilities that commited that violation as values
        vioDict = {}
        # Sort the violation codes for easier reading for user
        sortedCodes = sorted(self.dataDict['violation codes'])
        for v in sortedCodes:
            resultsInspections = collection_vendors.find({'VIOLATIONS.VIOLATION CODE':v}, {'FACILITY NAME':1})
            # Hash map to see if facility has been included - only need ONE instance of violation per facility
            facilityNames = {}
            for r in resultsInspections:
                # if facility name has already been counted, continue to next loop
                if r['FACILITY NAME'] in facilityNames:
                    continue
                # else, facility name has not yet been counted, add appropriately
                else:
                    if v in vioDict:
                        vioDict[v] = vioDict[v] + 1
                    else:
                        vioDict[v] = 1

        
        # Display violations vs no. of establishments
        vioKeys = list(vioDict.keys())
        vioVals = list(vioDict.values())
        # Create 4 subplots using slices of the keys & values lists
        fig,a =  plt.subplots(2,2)
        a[0][0].bar(vioKeys[:round(len(vioKeys)*0.25)], vioVals[:round(len(vioKeys)*0.25)])
        a[0][1].bar(vioKeys[round(len(vioKeys)*0.25):round(len(vioKeys)*0.5)], vioVals[round(len(vioKeys)*0.25):round(len(vioKeys)*0.5)])
        a[1][0].bar(vioKeys[round(len(vioKeys)*0.5):round(len(vioKeys)*0.75)], vioVals[round(len(vioKeys)*0.5):round(len(vioKeys)*0.75)])
        a[1][1].bar(vioKeys[round(len(vioKeys)*0.75):], vioVals[round(len(vioKeys)*0.75):])
        # Show in console as well as window
        plt.show()

        # creating the Tkinter canvas containing the Matplotlib figure 
        canvas = FigureCanvasTkAgg(fig, master = f2)   
        canvas.draw() 

        # grid the canvas on the Tkinter window 
        canvas.get_tk_widget().grid(row=0) 
        
    def createCorrelationsPlot(self, f3):
        # Find number of violations per vendor, and their zip code
        insViosDict = {'Facility ID': [], 'Violations quant': [], 'Zip': []}
        insVios = collection_vendors.find({'_id':{'$regex':'INS-'}}, {'FACILITY ID':1,'VIOLATIONS':1, 'FACILITY ZIP':1})
        # Create arrays for each key in dictionary (use sentinel NA if value does not exist)
        for r in insVios:
            if 'FACILITY ID' in r:
                insViosDict['Facility ID'].append(r['FACILITY ID'])
            else:
                insViosDict['Facility ID'].append(NA)
            if 'VIOLATIONS' in r:
                insViosDict['Violations quant'].append(len(r['VIOLATIONS']))
            else:
                insViosDict['Violations quant'].append(NA)
            if 'FACILITY ZIP' in r:
                insViosDict['Zip'].append(r['FACILITY ZIP'])
            else:
                insViosDict['Zip'].append(NA)
                
        #  Create DataFrame - sorted by zip to make visualisations easier to understand
        insViosDF = pd.DataFrame(insViosDict)
        insViosDF.sort_values(by=['Zip'],inplace=True)

        # Creating the figure and Seaborn plot
        figure = Figure(figsize=(6, 6))
        ax = figure.subplots()
        sns.scatterplot(x="Zip", y="Violations quant", data=insViosDF, ax=ax)
        
        # create the Tkinter canvas 
        canvas = FigureCanvasTkAgg(figure, master = f3)   
        canvas.draw() 

        # placing the canvas on the Tkinter window 
        canvas.get_tk_widget().grid(row=1) 
    
    def calcAverages(self):
        calcCatVal = self.calcCategory.get()
        calcYearVal = self.calcYear.get()
        calcAv = AveragesCalculator(calcYearVal, calcCatVal)
        avResults = calcAv.zipScores()
        self.averagesResultsFrame(avResults, calcYearVal, calcCatVal)
        
    def averagesResultsFrame(self, resultsAv, calcYearVal, calcCatVal):
        avResultsWindow = Tk()
        avResultsWindow.title(f'Averages Results for {calcCatVal} in {calcYearVal}')
        avResultsWindow.geometry("600x400")
        avResultsWindow.columnconfigure(0, weight=1)
        avResultsWindow.rowconfigure(0, weight=1)
        mainframe = ttk.Frame(avResultsWindow, padding="16 16 16 16")
        mainframe.place(relheight=1, relwidth=1)

        tree = ttk.Treeview(mainframe)
        tree.place(relheight=1, relwidth=1)
        treescrolly = Scrollbar(mainframe, orient='vertical', command=tree.yview)
        treescrollx = Scrollbar(mainframe, orient='horizontal', command=tree.xview)
        tree.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set)
        treescrolly.pack(side='right', fill='y')
        treescrollx.pack(side='bottom', fill='x')

        tree["columns"]=("one","two","three")
        tree.column("#0", width=75, minwidth=75)
        tree.column("one", width=75, minwidth=75)
        tree.column("two", width=75, minwidth=75)
        tree.column("three", width=75, minwidth=75)

        tree.heading("#0",text="Seating")
        tree.heading("one", text="Mean")
        tree.heading("two", text="Median")
        tree.heading("three", text="Mode")

        for index, row in resultsAv.iterrows(): 
            tree.insert('','end', text=index, values=[row['mean'],row['median'],row['mode']])
        print(f'This is resultsAv: \n{resultsAv}')
 

In [ ]:
ui = UserInterface('summative')
ui.createWindow()

/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/Inspections.csv
{'inspections': '/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/Inspections.csv', 'inventory': '', 'violations': ''}
/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/Inventory.csv
{'inspections': '/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/Inspections.csv', 'inventory': '/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/Inventory.csv', 'violations': ''}
/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/violations.csv
{'inspections': '/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/Inspections.csv', 'inventory': '/Users/ericagoodwin/Documents/york-msc/Module 8 - Advanced Programming/Week 6/training_data/Inventory.csv', '

DA0W5LPRX
DA0WCD49W
DA0WDAEDB
DA0YYPK8Y
DA1192561
DA1316717
DA1OZLKPX
DA1PJHSIN
DA2NSO1MK
DA3840ZZK
DA3DW2OO4
DA3JXKMPY
DA3UFQVYT
DA3XQEIMQ
DA44JPXVR
DA4JO5RQT
DA4VIZSBG
DA5RBJ0IA
DA5WI2FV5
DA6PKYHUP
DA6YSPGRK
DA7AJYL0N
DA7DH9LUD
DA7H8UHYB
DA7LDBG40
DA7ONYDBV
DA7Q0J99K
DA7WWDGIN
DA84PLLEZ
DA88HOFIT
DA8BD5RCA
DA8MFJIPF
DA8MRVZ25
DA9G8NK2Y
DA9T1PKUL
DAA6N48XO
DAAIVUBGF
DAAJV3OKS
DAAUREMDL
DAAW6AZ85
DAB00GDR8
DAB0JJ647
DABB2PF0U
DABJA0SWU
DABJOSJXA
DABOXI5MC
DABPBB0IG
DABRECGWQ
DABVPECX1
DABW7WRSY
DACDU6PPL
DACGSG260
DACIEW1IU
DACJCQKPO
DACQNDFSG
DACWJZLHX
DACYMPJKQ
DADF5KFKP
DADG3RVSR
DADNUQ85E
DADV8NXXD
DAE02JTFE
DAECLHVYK
DAEGS5QZV
DAEKXHLPP
DAENST80O
DAEOMJPUQ
DAEUCK01S
DAEWFSZC4
DAF3AYENJ
DAFBCYBPY
DAFF3Y61Y
DAFFFRTOW
DAFLAJO3G
DAFTPWPOE
DAFWCRPPX
DAG0J1ACY
DAG83WHBD
DAGHYOZOJ
DAGI526AN
DAGJMCOUR
DAGM840TZ
DAGOFMOPU
DAGWYJHKE
DAGYIDEYH
DAH0HY2IT
DAH5VK4DQ
DAHEGBBBQ
DAHN7S1JO
DAHOTQAGK
DAHW62AFP
DAI0RYDEK
DAIADP1F7
DAIMXYQB1
DAIOVCMPG
DAITCPWRB
DAIU9NYTW
DAJ0GYHMQ
DAJ1JSDCX
DAJ4QOM2O


DAELW89OT
DATYO9YLQ
DA9C0PFIC
DASPRLDHZ
DA15K1VLJ
DA3K640UF
DADVMOOJR
DAKR05EWQ
DAWCYGABM
DAX7VAY9G
DADQU8SO4
DADUUDF3M
DAD1WFDH3
DAVOAOOE1
DAXOONLTN
DA0H2507C
DAP7JWKHT
DAZKVDUNJ
DA1VCN7FB
DAAMJYSCG
DA0U8WKG3
DA6EIL9UX
DA6FSPBZE
DAASC45SP
DABS0N6GC
DABY8BAQG
DAEVGUIIQ
DA7YVOQPZ
DAANTR0AU
DAKDK0N04
DALFW703H
DAN8X0CAT
DAQ3SUOJV
DA13TSDHU
DAUX1O0HA
DA00L4URX
DAAK0W6AR
DAR0OPKDE
DARVY4I0J
DABGFP2QM
DAHGN0ESC
DALOXJ08W
DA5MBDJE4
DAA1RYZX2
DAKKYNFG6
DAJCNIWRB
DAO7AU9MW
DAWQKTZKY
DAQBUPYNA
DAX4BANOX
DA0WY2L80
DA2IEGUQH
DA40SFRWM
DAAV0UGRB
DAES1RN4K
DATD6EFOZ
DAUH8A00L
DAV0CTFAI
DAVKRF08L
DAYB7JXA2
DA0E60JTR
DA1EYID30
DA5AM06RC
DAGT0DF1X
DAGUTDSJE
DAGVC9CFP
DAL2S0FTU
DAQEUUWW2
DAZT7GMF9
DA0ITBXEW
DA5L0DSR1
DAJDLRYLM
DAJRIWAHN
DAJTZRYRR
DANBFNOH1
DASAYZUGR
DA8MTSY8V
DA9WUWG1P
DAWK2W08T
DAWOUH6YG
DALWDJ6HC
DAN6XBGDS
DANEUHE07
DANMLMXDM
DAQOCS37N
DAS9WKA40
DAT7DWRGR
DAZ60Z1Q0
DA0HGD0WB
DA2JDJLKV
DA8C00M1O
DABOT3OK1
DAD4IO7M7
DATNKUJX1
DA0AWXQR1
DA1ZA0Z7I
DAELSHAI9
DAGU2GOAN
DAO45DBRM
DARRZH10Q


DAVEL50TU
DAVF1FCMD
DAVILSKZH
DAVJYWCJL
DAVNB2KDM
DAVQ5JJAY
DAVQON0NS
DAVSNTPOW
DAVVGH1UO
DAVWTEXT7
DAVXCN0JZ
DAVXY1UWU
DAW0NVKO1
DAW1D9GAS
DAW3IG3XD
DAWB0BPKX
DAWCHUUAC
DAWFIAIR8
DAWHOHBS7
DAWK6B7V9
DAWO9NZY0
DAWQTAOM5
DAWSCGLYP
DAX6ATGMA
DAXBAFOSW
DAXCFMMKU
DAXEH5BBO
DAXFV90WF
DAXHBHKHQ
DAXHJD6IU
DAXK0FBMU
DAXKJ20JZ
DAXMGDTJF
DAXPEXENX
DAXQKNFB3
DAXS0HUEV
DAXTJJQTZ
DAXTQPA1X
DAXTXWAMW
DAXXD0HBX
DAY8JHF25
DAYAHEBQP
DAYBTMYRV
DAYCKPQIO
DAYDGJA09
DAYFIUMIA
DAYJQPOEN
DAYKJTAOK
DAYOBSKRD
DAYR0DXBT
DAYSZGABI
DAYW5BUGR
DAYW5MFQB
DAYYR6FA3
DAZ0Y0RLK
DAZ1MX1F1
DAZ2UJ2DQ
DAZGQM7XG
DAZKLGAH1
DAZKMNEX2
DAZKWB9P4
DAZMLHGHB
DAZOOXXW0
DAZOQKRXY
DAZQRIRR1
DAZR4JPVL
DAZTQFFBU
DAZYCVZVV
DAZZD8OJR
DAZZTCR8W
DA9O6RM4R
DANXX1QDZ
DAOM01OYZ
DAA00MHOO
DAGWGS5A4
DAZUV721U
DAE9UVOS0
DAQPKZDQ8
DA7O0KRJJ
DA9080BLP
DAEDHOCP3
DAHPFVOKV
DAQY60QJN
DAGKRFGLE
DAQBKZ1GJ
DABCAGABK
DAMBQ3Q34
DAUMA6MYR
DAY00LU0U
DAYGILM1P
DAV9CA30H
DATCD6B2C
DAHJG77UO
DATPJFJAA
DA7I3SGQ0
DABT23WTH
DAEAB1GOV
DAGVN3VAF
DAGVQOI72
DANHRW3R7


DA50ZMLQA
DA51FC9NM
DA57QFIJV
DA58EHFNG
DA58HYOHZ
DA5ALQPVV
DA5AQL7AC
DA5BR8WNF
DA5EI6LAV
DA5EMH8XP
DA5MPIHAT
DA5MWCVP4
DA5NIPZEY
DA5OT6BEF
DA5OW2Z8F
DA5UHMWTL
DA5Y92ZAG
DA6AGF1ZS
DA6D0GTTG
DA6DBDWUQ
DA6E1SQ7W
DA6EMH2G5
DA6EWXRTO
DA6I0H70G
DA6LHQWQ4
DA6NZ8NJA
DA6P9JTCH
DA6T4M3CG
DA6VU4RQB
DA6XNOY0U
DA708DZX0
DA7HLQJ0I
DA7JS0HLS
DA7KRFM2X
DA7LRIIHK
DA7WGDJEU
DA80QPYHQ
DA85PPOAU
DA890LO2M
DA8BPHWRV
DA8HNUR0M
DA8IYDZMG
DA8MIRQCU
DA8NYYKBL
DA8YRK57D
DA8Z6LUNH
DA90PQBQU
DA93HY4F2
DA9CWQFWZ
DA9D10XXH
DA9L6WKVO
DA9UIU6LT
DAA0GWVAJ
DAAAP59VH
DAACSBHRT
DAAFEY7RC
DAAGGEOHP
DAAHNTTYV
DAAJINIKC
DAAK5WSRD
DAAPH73T0
DAAQWVJM0
DAARE7ZPK
DAASEAOPO
DAAWXHV7T
DAAXSWLNB
DAAZZOBBI
DAB0DSIBS
DAB91FCQQ
DABC3GQHC
DABCWZFK9
DABDFJW07
DABDLP9JX
DABERXE8T
DABHKRDC0
DABJ710UP
DABJG40ZB
DABLDULFG
DABLI7RK4
DABMVYFK5
DABN2C0VN
DABNK035O
DABPR0KBE
DABQEUJSM
DABT1K6HW
DABYUSNCN
DAC0IVTFK
DAC1P9PLP
DAC65WCVZ
DAC70XP0D
DAC9BGXTQ
DACHGYGMM
DACQCGLGV
DACS0UZYY
DACT242ND
DACTMFMK2
DACWOFW5F
DACXMHHM7
DAD0BHZYH
DAD7SLJ1D


...

DAIAU4QV6
DAMSTPNJH
DAOTI7XSI
DAPDIVSHP
DAXZOXKDX
DAYGCVCAN
DA0L1QOII
DA4PLUFJC
DA6CVHKNT
DABXJNYVQ
DAGT6ICEH
DALAES7I8
DASIH3RTW
DA0IVFG8I
DA0KQMQCJ
DA1294966
DA71SNND0
DA7FFUL2U
DA8UUSQCL
DA8V9YPFY
DAAU3LHC0
DACF5VSCA
DACQ21LQ3
DACX2LKMR
DAD6T0TCU
DADCUV8RP
DADEQLZTK
DAHIDUDWJ
DAIDNEYRM
DAJUT9LRW
DAKF3IRUD
DALHJCTG2
DAMVZENIG
DAMYDFYHD
DANBD7VJK
DANH6IJJA
DANO22E1Y
DAOUBOLD2
DAOZWBFJY
DAP3JS2SL
DAPI70130
DAPVTFXXT
DAQ0YZXPA
DAQ8LTYZS
DAQPYEQNM
DARBKV20M
DARST1SGI
DARV45ABN
DASH5ICAF
DASIUTMLH
DAVQUTPT2
DAXEZ1Y8L
DAZGW10KT
DAZY2MJY0
DAIV8JPB0
DANUMPJPH
DAQZXRUM8
DATZFGZWW
DAVMBZ6JZ
DA0830279
DA0N5DZA1
DA6PL6UYM
DAD7UKFX6
DAIZYE4QR
DAMRQ7VOR
DAD3CZQLQ
DAE5YB0OA
DAFNDJ8CW
DAJ2WBD08
DAKKENI0O
DASOD0KVX
DAUHKQSZQ
DA0FOGDAQ
DA0JRJSNC
DAIXBOZ0F
DAPSVP3NT
DAQEGWURA
DAS3LUCNR
DAT0ECQZ3
DAWJNBOVR
DADJW950D
DAJC8KBO6
DAOUZKZBF
DAW18L7F6
DAFUMOOW3
DAOVNBWXW
DAWHF09IU
DAXEVR1SK
DAZCIBH9X
DAPN05AV9
DABRQCIZC
DASNICENP
DAPUWMQJO
DA1TNCGY0
DA8RETMEW
DAA6PPWLE
DACWRTO7G
DA0TPC5XY
DA70V0LH6
DAM0FEQQA


DAXXMG4YE
DA0HS9WFW
DA9OFV8MO
DAANW007B
DADRQGOYH
DALCL9BTR
DAMMQLOAW
DAQMLM76J
DA7IOAA8A
DABSHYMYJ
DAJNTIMLO
DALQ60LLQ
DAQLDJH5T
DAVLAJXE8
DAVN44O9E
DAZ0MNXBK
DABX237HC
DAFBEJAWZ
DAIXO9IQX
DANGWV64C
DAXQMIKCO
DAYKKAFRJ
DA4MDZB0A
DA6YBQAJF
DA94R1PLB
DABOWKCKX
DAH4OJR8S
DAPN3EY5V
DAWGMXJAF
DAWQIC8BY
DA1JL0UIO
DA22PYV3S
DABGWCKLA
DACJPIWCU
DAFYEO64N
DAN0SOV2F
DAUSW93NM
DA8DAW55V
DABPQ3EFE
DAC8WNFXO
DAEJVLNCY
DAM36QB6B
DASDCLJDY
DA0SDPEVI
DAGDMV13R
DAGKA0IFJ
DAGTVITTN
DAS3A0JAA
DAVLZBDQB
DADLWIOS5
DAYVW9WYV
DAZDGWH5F
DACPCRVQ0
DAEDBXPUB
DAF9MJ0YF
DAJZTMUKG
DASQ2TWP6
DAWY5D9CI
DAYOX9J3U
DA0BCP3OJ
DA47YVBKT
DAE4ADDB4
DAL6XTV2E
DALPBTCRW
DARWMQVTI
DAWWK792O
DAYMLQMMM
DA2NB28E2
DAJAWAVTP
DAJNUCCZZ
DA2RKXNQ6
DA9VVOVKH
DAGEIV03S
DAH9WIJ01
DAMPEPDWB
DARFFMXZL
DAIFUKSWI
DAK3AMGA7
DAM4Z4S26
DAOIAUKUK
DAPE8HM2B
DAQBW89W0
DAUGXXT7R
DAZAXSC6K
DAZZWZQER
DA1225423
DAAEJKFOX
DABQWERBO
DACFFQUNK
DAFP4RDVO
DALSL2BHO
DALUW0RMM
DAR0UGUVO
DAAFLY0I4
DADX4LJ60
DAJFGL1ZN
DALU6BPAW
DARC3NYHP
DA6QKLXAN
DAABN1WE2


DATGDRVBT
DATGMKHSI
DATGQ3VQH
DATINYKSZ
DATJLE0OR
DATJSEVOI
DATJUW0W0
DATLFHCT6
DATLGTWRG
DATLUXU0B
DATNCWVOM
DATNQC8RA
DATNXOUDR
DATQME6ZU
DATSR2ONE
DATSUUB1M
DATSWPZDH
DATY0RHFV
DATYKC9GP
DATYMT05A
DAU02ARJC
DAU2IURFK
DAU8MZZR0
DAUAKDZXP
DAUBMIVYE
DAUC5PALN
DAUE9IDEM
DAUEHQ2J7
DAUEVUBXW
DAUG1FYQQ
DAUGIS3D4
DAUHYGDOM
DAUJMJWDF
DAUNART4A
DAUOXEUYV
DAUPU47QX
DAUPX99SA
DAUREXQZJ
DAUT5VMLN
DAUUJUFVQ
DAUVHI96J
DAUW87KK3
DAUWG7BWZ
DAV00EXEJ
DAV0XHSK1
DAV69CNCV
DAV78A1VW
DAV7VGXZJ
DAV8EJXCI
DAVA9LZCR
DAVAET38I
DAVEZW0MW
DAVFGK3OL
DAVGSSZ48
DAVHNVPBW
DAVIAFLEI
DAVIF5SGG
DAVIYQLX4
DAVJ5XJMU
DAVJOZTSE
DAVKDX302
DAVLZ6VPS
DAVM14EHY
DAVMOIQOR
DAVMRPQ5Q
DAVMZ2LK1
DAVO09Y3N
DAVP0DEVY
DAVQIWQLF
DAVSMFGY5
DAVSX21DJ
DAVUASTTR
DAVUDVRLA
DAVWO6M4G
DAVWS2EXX
DAVX2YCQE
DAVZDR1WB
DAVZHZDD9
DAVZW08OJ
DAW3BFK98
DAW5MSG0D
DAWB0AUDY
DAWBGH6UL
DAWC0TUIV
DAWDEHYH9
DAWE6LQVM
DAWEN49KM
DAWF95P0D
DAWFL3BEA
DAWGIYIOK
DAWH2XA2F
DAWINLWAS
DAWIPIX6O
DAWM3RLFE
DAWMN0M0F
DAWNIXDNV
DAWNUG0MB
DAWOAVMMS
DAWODYPRT
DAWPK9J5P


DALRAV3WF
DAMYULSGZ
DAR2TCX16
DARLPNDCR
DA4Y2GLR1
DA76XCZAC
DABHLTFW3
DAM0D95ZD
DAN6NJLQE
DAO6XZ7XZ
DAODSUXYV
DAQZZJRIU
DAFPN81GM
DAMT2OFJD
DATFYX3YV
DA0953667
DAJDPUIVF
DAOL67KBQ
DAPD6ZSX5
DA8KDELFD
DA9I4KBWK
DA9YXLFPC
DAEQSKEBU
DAF5NSGLL
DALGQJQ7D
DAXARDI00
DA92XCZGT
DAINPRPDN
DAZULR6KL
DA11G6ZZA
DAIN18YCO
DALGRSQL0
DALWECP0X
DAMMVJV1J
DAQZTPXGG
DARO00ID7
DA8BGBLEM
DAECXYXDY
DAETXRDDJ
DALACIVS4
DANX7JWFJ
DAVDND2SJ
DAZAATHKI
DAANFGCOF
DAFIWRKYV
DAFMEEBGY
DAM6HMBBD
DAO02L6SB
DA1G0I78A
DAH9MMRZ7
DAJH1HHYV
DAP0O0J15
DAQXGNRP3
DATENG4HQ
DAXCDNMBQ
DA0HPPNHH
DA3M2YMRT
DACGAVVVW
DAG0LB0JI
DAGJI0HSO
DAIFS5WX8
DAJ2RGOL1
DAKKMXC85
DAN0ATPKP
DARGYFCPC
DAULESINN
DAVAKTWMJ
DAWXQCGES
DAXCKEEMT
DAADWIFRR
DA046BAH1
DAQQ9NN9H
DA0DFSD5U
DAG5XXOP0
DARB7S0TP
DAZD7BGMA
DAGJR7ZEW
DAJBXEISY
DAP8AIUB2
DAPHBANK0
DATZS37PO
DAWP2AGKY
DA7PK65BN
DAAOIIK5J
DANS5SLNB
DATR0WT0Y
DAUN050EG
DA5ZQI6KK
DAHYY1P06
DAK5HB9A6
DAKKKUAGM
DAKSNLHYM
DAKUGUF0H
DA04LFAGE
DAKHKIVTY
DARFCCUNN
DAZA00FC7
DAOSLWB1Q
DAVQFMO8T
DAGLYKTS0


DAGLVPWJK
DAPMIUL8L
DATEYZHLK
DATJA9AXY
DAZE0AZE4
DAZXISWD3
DA9O9WUJS
DAHRYRUOF
DAPJEXB19
DAQYIVCYZ
DASYYPEEH
DASZOTUG5
DAU6WPYHH
DA0MZPADF
DAG07FTBR
DAIITYMHC
DAMSMSCTA
DAQJHFSFP
DARDGSA2E
DAN8JG1HI
DAYXFCSHB
DA0C3A7US
DA1SDF50W
DAMKWJQML
DASRVRTOL
DAUMJIRIQ
DAYJQ50P6
DA8LVFP8X
DAJQDO7YW
DAPLWD0U0
DAQYCAQAZ
DAXATQLRO
DAXEUDXOI
DA2W89VGJ
DALR7BCTZ
DAOGKUBL0
DAYWUE0SZ
DA6NEAHGO
DAC7J479I
DAKWYERWG
DAMENLAQN
DANS3UFL6
DA0708245
DA8TXLEGF
DA9J6YWZF
DAA0KRGIH
DABMIN9B0
DACBIZFPY
DADLOFOE4
DAETAH7F5
DAG0DS9F0
DAG4IJPKY
DAJTRAWPF
DAM5HFIAV
DAMEAZGGM
DAMYYBWL4
DANBWVMCB
DANGRO6IP
DAO2OYVRT
DAOH7GFB9
DAP0WKBBY
DAPW6V0DO
DARCKBF95
DAUQDW3W8
DAW0MRQTL
DAWLDOXXJ
DAXEQCPEP
DAXTKCOGN
DAIFNXKVG
DAW2U0CNZ
DAAKSLLYK
DAF6WOCGR
DAKCCRQTP
DAKH2KIE9
DAQYN9MEI
DAHUM0YLL
DAND87MCT
DAO1AKPUD
DAPZDEOVO
DAQMMXN10
DATUKL3J2
DAZMVCYKX
DA2FU8OLR
DACQXEX80
DAEYKMXL0
DAGHTAKWV
DAWHQX5AH
DADSSB00N
DALBFW00P
DASHGDSAG
DA0N08ASU
DA8P2ZWMU
DAHMMAHAZ
DAJYM5QDK
DANX0DBUO
DAVX0RYHN
DAX81YOPH
DA0920700
DA4CJ8ORB
DA5SWAZHO
